### Sample Waterfall



A simple waterfall with actions in different places

In [2]:
sampleWaterfall = {
    "amortizing":[
         ["calcAndPayFee","acc01",['trusteeFee']]
         ,["accrueAndPayInt","acc01",["A1"]]
         ,["payPrin","acc01",["A1"]]
         ,["payPrin","acc01",["B"]]
         ,["payIntResidual","acc01","B"]
     ],
    "endOfCollection":[["calcAndPayFee","acc01",['serviceFee']],
                      ],
    
    "cleanUp":[["sellAsset", ["Current|Defaulted", 1.0, 0.2],"acc01"]
              ,["payIntResidual","acc01","B"]
              ],
    
    "closingDay":[["calcAndPayFee","acc01",['issuanceFee']],
                 ]
    }

Plug in the waterfall into the deal object

In [3]:
from absbox import Generic

test01 = Generic(
    "TEST01"
    ,{"cutoff":"2021-03-01","closing":"2021-06-15","firstPay":"2021-07-26"
     ,"payFreq":["DayOfMonth",20],"poolFreq":"MonthEnd","stated":"2030-01-01"}
    ,{'assets':[["Mortgage"
        ,{"originBalance":2200,"originRate":["fix",0.045],"originTerm":30
          ,"freq":"Monthly","type":"Level","originDate":"2021-02-01"}
          ,{"currentBalance":2200
          ,"currentRate":0.08
          ,"remainTerm":30
          ,"status":"current"}]]}
    ,(("acc01",{"balance":0}),)
    ,(("A1",{"balance":1000
             ,"rate":0.07
             ,"originBalance":1000
             ,"originRate":0.07
             ,"startDate":"2020-01-03"
             ,"rateType":{"Fixed":0.08}
             ,"bondType":{"Sequential":None}})
      ,("B",{"balance":1000
             ,"rate":0.0
             ,"originBalance":1000
             ,"originRate":0.07
             ,"startDate":"2020-01-03"
             ,"rateType":{"Fixed":0.00}
             ,"bondType":{"Equity":None}
             }))
    ,(("issuanceFee",{"type":{"fixFee":30}})
      ,("serviceFee",{"type":{"annualPctFee":[("poolBalance",),0.02]}})
      ,("trusteeFee",{"type":{"annualPctFee":[("poolBalance",),0.01]}})
      )
    ,sampleWaterfall
    ,[["CollectedInterest","acc01"]
      ,["CollectedPrincipal","acc01"]
      ,["CollectedPrepayment","acc01"]
      ,["CollectedRecoveries","acc01"]]
    ,None
    ,None
    ,None
    ,None
    ,("PreClosing","Amortizing")
    )

Adding a `call` assumption to trigger the `clean up` action in the waterfall

In [21]:
from absbox import API,EnginePath
localAPI = API(EnginePath.DEV,check=False)

r = localAPI.run(test01
               ,poolAssump = None
               ,runAssump = [("call",("poolBalance",300))]
               ,read=True)

Warning Message from server:

### On Closing Day

The action only performs on `Closing Day` for deal with `PreClosing` status

In [6]:
test01.dates

{'cutoff': '2021-03-01',
 'closing': '2021-06-15',
 'firstPay': '2021-07-26',
 'payFreq': ['DayOfMonth', 20],
 'poolFreq': 'MonthEnd',
 'stated': '2030-01-01'}

Action to be performed on `Closing Day`

In [8]:
sampleWaterfall['closingDay']

[['calcAndPayFee', 'acc01', ['issuanceFee']]]

Transaction log from the account 

In [35]:
r['accounts']['acc01'].loc['2021-06-15']

,balance,change,memo
date,,,
2021-06-15,0.00,0.00,<Pool:CollectedRecoveries>
2021-06-15,0.00,0.00,<Pool:CollectedPrepayment>
2021-06-15,268.63,268.63,<Pool:CollectedPrincipal>
2021-06-15,324.60,55.97,<Pool:CollectedInterest>
2021-06-15,324.60,0.00,<SeqPayFee:serviceFee>
2021-06-15,294.60,-30.00,<SeqPayFee:issuanceFee>


### Pool Collection Days

In [10]:
test01.dates

{'cutoff': '2021-03-01',
 'closing': '2021-06-15',
 'firstPay': '2021-07-26',
 'payFreq': ['DayOfMonth', 20],
 'poolFreq': 'MonthEnd',
 'stated': '2030-01-01'}

The action in the `endOfCollection` will be performed each time when collecting proceeds from the pool

The dates were described by `poolFreq`

In [11]:
sampleWaterfall['endOfCollection']

[['calcAndPayFee', 'acc01', ['serviceFee']]]

In [34]:
r['accounts']['acc01'].loc['2021-07-31']

,balance,change,memo
date,,,
2021-07-31,0.00,0.00,<Pool:CollectedRecoveries>
2021-07-31,0.00,0.00,<Pool:CollectedPrepayment>
2021-07-31,68.28,68.28,<Pool:CollectedPrincipal>
2021-07-31,81.15,12.87,<Pool:CollectedInterest>
2021-07-31,77.87,-3.28,<SeqPayFee:serviceFee>


### Distribution Day

In [12]:
test01.dates

{'cutoff': '2021-03-01',
 'closing': '2021-06-15',
 'firstPay': '2021-07-26',
 'payFreq': ['DayOfMonth', 20],
 'poolFreq': 'MonthEnd',
 'stated': '2030-01-01'}

The action in the `amortizing` will be performed each time on the dates were described by `payFreq`

In [13]:
sampleWaterfall['amortizing']

[['calcAndPayFee', 'acc01', ['trusteeFee']],
 ['accrueAndPayInt', 'acc01', ['A1']],
 ['payPrin', 'acc01', ['A1']],
 ['payPrin', 'acc01', ['B']],
 ['payIntResidual', 'acc01', 'B']]

In [15]:
r['accounts']['acc01'].loc['2021-08-20']

,balance,change,memo
date,,,
2021-08-20,77.61,-0.26,<SeqPayFee:trusteeFee>
2021-08-20,74.18,-3.43,<PayInt:A1>
2021-08-20,0.00,-74.18,<PayPrin:A1>
2021-08-20,0.00,0.00,<PayPrin:B>
2021-08-20,0.00,0.00,<PayYield:B>


### Clean Up

The `clean up` waterfall only perform when the deal met the conditin from `runAssump` assumption

In this case, the `clean up` happens at date `2023-06-20`,we can inspect that via `result``status`

In [25]:
r['result']['status']

,Date,From,To
0,2021-06-15,PreClosing,Amortizing
1,2023-06-20,Amortizing,Called
2,2023-06-20,DealEnd,Clean Up


We are able to identify the actions via account transaction

In [29]:
sampleWaterfall['cleanUp']

[['sellAsset', ['Current|Defaulted', 1.0, 0.2], 'acc01'],
 ['payIntResidual', 'acc01', 'B']]

In [24]:
r['accounts']['acc01'].loc['2023-06-20']

,balance,change,memo
date,,,
2023-06-20,80.52,-0.09,<SeqPayFee:trusteeFee>
2023-06-20,80.52,0.00,<PayInt:A1>
2023-06-20,80.52,0.00,<PayPrin:A1>
2023-06-20,78.51,-2.01,<PayPrin:B>
2023-06-20,0.00,-78.51,<PayYield:B>
2023-06-20,240.24,240.24,<Liquidation>
2023-06-20,0.00,-240.24,<PayYield:B>


### Summary Via `[result][waterfall]`

There is a function that would summaries which waterfall actions are being executed by dates

In [31]:
r['result']['waterfall']

,Date,Waterfall Location
0,2021-06-15,<EndOfPoolCollection>
1,2021-06-15,<OnClosingDay>
2,2021-06-30,<EndOfPoolCollection>
3,2021-07-26,<DistributionDay:<Amortizing>>
4,2021-07-31,<EndOfPoolCollection>
5,2021-08-20,<DistributionDay:<Amortizing>>
6,2021-08-31,<EndOfPoolCollection>
7,2021-09-20,<DistributionDay:<Amortizing>>
8,2021-09-30,<EndOfPoolCollection>
9,2021-10-20,<DistributionDay:<Amortizing>>
